# LZ78 Sequential Probability Assignment: Python Interface for Rust Implementation
This code is associated with the paper [A Family of LZ78-based Universal Sequential Probability Assignments](https://arxiv.org/abs/2410.06589).

The codebase is in Rust, with Python bindings. This tutorial goes through how to use the Python API; if you are familiar with Rust or want to learn, feel free to look at `crates/lz78` for the source code and `crates/python` for the bindings (the former is well-documented, whereas the latter is not-so-well-documented).

## Setup
You need to install Rust and Maturin, and then install the Python bindings for the `lz78` library as an editable Python package.
1. Install Rust: [Instructions](https://www.rust-lang.org/tools/install).
    - After installing Rust, close and reopen your terminal before proceeding.
2. If applicable, switch to the desired Python environment.
3. Install Maturin: `pip install maturin`
4. Install the `lz78` Python package: `cd crates/python && maturin develop && cd ../..`

You also need the `lorem` ([context](https://loremipsum.io/)), `numpy`, and `requests` Python packages for this tutorial.

### Setup Notes
If you are modifying the Rust code and are using VSCode, you have to do a few more steps:
1. Install the `rust` and `rust-analyzer` extensions.
2. Adding extra environment variablers to the rust server:
    - In a terminal, run `echo $PATH`, and copy the output.
    - Go to `Preferences: Remote Settings (JSON)` if you are working on a remote machine, or `Preferences: User Settings (JSON)` if you are working locally (you can find this by pressing `F1` and then searching), and make sure it looks like the following:
        ```
        {
            "rust-analyzer.runnables.extraEnv": {
                "PATH": "<the string you copied in the previous step>"
            },
        }
        ```
3. Open `User Settings (JSON)` and add `"editor.formatOnSave": true`
4. Restart your VSCode window.


## Imports

In [1]:
from lz78 import Sequence, LZ78Encoder, CharacterMap, BlockLZ78Encoder, LZ78SPA
from lz78 import encoded_sequence_from_bytes, spa_from_bytes
import numpy as np
import lorem
import requests
from sys import stdout
from os import makedirs

### Notes
- Sometimes, Jupyter doesn't register that a cell containing code from the `lz78` library has started running, so it seems like the cell is waiting to run until it finishes. This can be annoying for operations that take a while to run, and **can be remedied by putting `stdout.flush()` at the beginning of the cell**.
- For a description of all classes and functions, got to `crates/python/lz78.pyi`. The docstrings there are the same as the ones that appear when you hover over a class or method in Jupyter/most IDEs.

## 1. Sequences

Any sequence of data that can be LZ78-encoded (i.e., a list of integers or a String) is represented as a `Sequence` object.
Storing sequences as this object (as opposed to raw lists or strings) allows for a common interface that streamlines the LZ78 encoding process.

Each sequence is associated with an alphabet size, A.

If the sequence consists of integers, they must be in the range ${0, 1, ..., A-1}$.
If $A < 256$, the sequence is stored internally as bytes.
Otherwise, it is stored as `uint32`.

If the sequence is a string, a `CharacterMap` object maps each character to a number between 0 and A-1.
More on this later.

**Inputs**:
- data: either a list of integers or a string.
- alphabet_size (optional): the size of the alphabet.
    If this is `None`, then the alphabet size is inferred from the data.
- charmap (optional): A `CharacterMap` object; only valid if `data` is a string.
    If `data` is a string and this is `None`, then the character map is inferred from the data.

The methods available for a `Sequence` object are described below.

### 1.1 Example: Integer Sequence

In [3]:
data = np.random.randint(0, 2, size=(10_000_000,))
int_sequence = Sequence(data, alphabet_size=2)

A limited number of Python list operations work on `Sequence`:

In [3]:
print(len(int_sequence))
print(int_sequence[-20:])

10000000
[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]


As a note, indexing a string-based sequence in this manner will return the integer-based representation of the string and not the string itself. You will have to use the corresponding character map to map these integers back to a string representation.

#### Instance method: `extend`

Adds data to the end of the sequence.
Data must be over the same alphabet as the current sequence.

In [4]:
more_data = np.random.randint(0, 2, size=(200,))
int_sequence.extend(more_data)

In [5]:
len(int_sequence)

10000200

#### Instance method: `alphabet_size`

In [6]:
int_sequence.alphabet_size()

2

#### Instance method: `get_data`
Returns the full sequence as an integer list or string.

In [7]:
extracted_data = int_sequence.get_data()
print(type(extracted_data))
print(extracted_data[-20:])

<class 'list'>
[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]


### 1.2 `CharacterMap`
A sequence is defined as integers from 0 to A-1, where A is the alphabet size, so we need a way to map strings to such integer-based sequences.

The `CharacterMap` class maps characters in a string to integer values in a contiguous range, so that a string can be used as an individual sequence.
It has the capability to **encode** a string into the corresponding integer representation, and **decode** a list of integers into a string.

Inputs:
- data: a string consisting of all of the characters that will appear in the character map. For instance, a common use case is:
    ```
    charmap = CharacterMap("abcdefghijklmnopqrstuvwxyz")
     ```

In [8]:
# generate some dummy data and make a character map
s = " ".join(([lorem.paragraph() for _ in range(10)]))
charmap = CharacterMap(s)

#### Instance method: `encode`
Takes a string and returns the corresponding integer representation.

In [9]:
charmap.encode("lorem ipsum")

[8, 7, 17, 11, 4, 5, 13, 1, 2, 3, 4]

It errors if any characters to be encoded are not in the alphabet.

In [10]:
# this should error, but with a helpful warning message!
charmap.encode("hello world")

RuntimeError: Character "h" not in mapping

#### Instance method: `filter_string`
Takes a string and removes any characters that are not present in the character mapping.
This is useful if you have some text with special characters, and you don't want the special characters to be in the alphabet.

In [11]:
charmap.filter_string("hello world. Lorem ipsum! @#$%^&*()")

'ello orld. Lorem ipsum '

#### Instance method: `decode`
Decodes an integer representation of a string into the string itself

In [12]:
charmap.decode(charmap.encode("lorem ipsum"))

'lorem ipsum'

#### Instance method: `alphabet_size`
Returns how many characters can be represented by the character mapping

In [13]:
charmap.alphabet_size()

32

### 1.3 Example: Character Sequence
A string-based sequence is sometimes referred to as a character sequence. It has the same interface as an integer sequence, except there is an underlying `CharacterMap` object that maps characters to corresponding integer values within the alphabet.

You can pass in a `CharacterMap` upon instantiation, or else the character map will be inferred from the data.

**Note**: if you pass in a `CharacterMap`, and the input string has characters not present in the character map, instantiation will error.
To avoid this, you can use `CharacterMap.filter` beforehand.

In [14]:
stdout.flush()
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ?,")
s = " ".join(([lorem.paragraph() for _ in range(1000)]))
charseq = Sequence(s, charmap=charmap)

Indexing a character sequence returns the integer representations of the corresponding characters.

In [15]:
print(charseq[100:130])

[17, 0, 53, 11, 0, 1, 14, 17, 4, 53, 13, 4, 16, 20, 4, 53, 0, 11, 8, 16, 20, 0, 12, 53, 12, 0, 6, 13, 0, 12]


#### Instance method: `get_character_map`
Returns the underlying `CharacterMap` object.
This will error if the sequence is not a character sequence.

In [16]:
charmap = charseq.get_character_map()
charmap.decode(charseq[100:130])

'ra labore neque aliquam magnam'

## 2. LZ78 Compression
The `LZ78Encoder` object performs plain LZ78 encoding and decoding, as described in "Compression of individual sequences via variable-rate coding" (Ziv, Lempel 1978).

### 2.1 `CompressedSequence` object
A `CompressedSequence` object stores an encoded bitstream, as well as some auxiliary information needed for decoding.
`CompressedSequence` objects cannot be instantiated directly,
but rather are returned by `LZ78Encoder.encode`.

The main functionality is:
1. Getting the compression ratio as `(encoded size) / (uncompressed len * log A)`,
    where A is the size of the alphabet.
2. Getting a byte array representing this object, so that the compressed
    sequence can be stored to a file

### 2.2 Example: LZ78 Encoding

In [17]:
# Make an input sequence to compress
stdout.flush()
data = " ".join(([lorem.paragraph() for _ in range(10_000)]))
charseq = Sequence(data)
encoder = LZ78Encoder()

#### `LZ78Encoder` Instance method: `encode`
Performs LZ78 encoding on an individual sequence, and returns a `CompressedSequence` object.

In [18]:
stdout.flush()
encoded = encoder.encode(charseq)

#### `CompressedSequence` Instance method: `compression_ratio`

In [22]:
encoded.compression_ratio()

0.3189317286014557

#### Saving a `CompressedSequence` object
`CompressedSequence` has functionality to produce a `bytes` object representation, which can be written directly to a file.
The function `encoded_sequence_from_bytes` produces a `CompressedSequence` object from this `bytes` representation.

In [24]:
stdout.flush()
bytes = encoded.to_bytes()

makedirs("test_data", exist_ok=True)
with open("test_data/saved_encoded_sequence.bin", 'wb') as file:
    file.write(bytes)

Now, let's read the compressed sequence from the file and decode it.

In [25]:
with open("test_data/saved_encoded_sequence.bin", 'rb') as file:
    encoded_bytes = file.read()
encoded = encoded_sequence_from_bytes(encoded_bytes)

In [26]:
stdout.flush()
decoded = encoder.decode(encoded)

In [27]:
assert decoded.get_data() == data

### 2.3 Block-Wise Compression
Sometimes, it might be useful to loop through blocks of data and perform LZ78 encoding on each block (e.g., if you need to do data processing before LZ78 compression and want to have some sort of pipeline parallelism).

The `BlockLZ78Encoder` has this functionality: you can pass in the input sequence to be compressed in chunks, and the output (`encoder.get_encoded_sequence()`) is as if the full concatenated sequence was passed in to an LZ78 encoder.

In [2]:
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ,?")

In [4]:
encoder = BlockLZ78Encoder(charmap.alphabet_size())

#### Instance method: `encode_block`
Encodes a block using LZ78, starting at the end of the previous block.

All blocks must be over the same alphabet, or else the call to `encode_block` will error.

In [5]:
stdout.flush()
for _ in range(1000):
    encoder.encode_block(Sequence(lorem.paragraph(), charmap=charmap))

In [13]:
# Oops, this won't work!
encoder.encode_block(Sequence([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))

RuntimeError: Expected String Sequence with character mapping ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '.', ' ', ',', '?'], got Byte (U8) Sequence with alphabet size 11

#### Instance method: `get_encoded_sequence`
Returns the compressed sequence, which is equivalent to the output of `LZ78Encoder.encode` on the concatenation of all inputs to `encode_block` thus far.

In [14]:
encoded_sequence = encoder.get_encoded_sequence()
encoded_sequence.compression_ratio()

0.3503199517726898

#### Instance method: `decode`
Decompresses the compressed sequence that has been constructed thus far.

In [30]:
stdout.flush()
decoded = encoder.decode()
print(decoded[376:400])
charmap.decode(decoded[376:400])

[20, 15, 19, 0, 19, 4, 12, 53, 21, 4, 11, 8, 19, 53, 13, 14, 13, 53, 12, 0, 6, 13, 0, 12]


'uptatem velit non magnam'

## 3. LZ78 Sequential Probability Assignment (SPA)
The `LZ78SPA` class is the implementation of the family of sequential probability assignments discussed in [A Family of LZ78-based Universal Sequential Probability Assignments](https://arxiv.org/abs/2410.06589), for Dirichelt priors.
In this section, `gamma` refers to the Dirichlet parameter.

Under this prior, the sequential probability assignment is an additive
perturbation of the emprical distribution, conditioned on the LZ78 prefix
of each symbol (i.e., the probability model is proportional to the
number of times each node of the LZ78 tree has been visited, plus gamma).

This SPA has the following capabilities:
- training on one or more sequences,
- log loss ("perplexity") computation for test sequences,
- SPA computation (using the LZ78 context reached at the end of parsing
    the last training block),
- sequence generation.

Note that the LZ78SPA does not perform compression; you would have to use
a separate BlockLZ78Encoder object to perform block-wise compression.

### 3.1 Example: LZ78 SPA on Markov Data

We will use the Markov probability source used in [(Rajaraman et al, 2024)](https://arxiv.org/pdf/2404.08335), where the transition probability depends solely on $x_{t-k}$.
Specifically, $x_t = x_{t-k}$ with probability $0.9$, and otherwise $x_t$ is picked uniformly at random from the rest of the alphabet.

The SPA works best when the alphabet size is $2$, but you can try out other alphabet sizes too.

First, we define some helper functions for generating the data (don't worry about understanding these; they are irrelevant to understanding the SPA itself).

In [90]:
# Helper methods for generating data; feel free run the cell without
# reading the code
def sample_index_from_dist(probabilities):
    cdf = np.cumsum(probabilities)
    cdf[-1] = 1 # in case of FP error
    return int(np.where(np.random.random() < cdf)[0][0])

def entropy(probs):
    return sum([-x * np.log2(x) for x in probs if x > 0])

def get_stationary_dist(transition_probabilities):
    eigvals, eigvecs = np.linalg.eig(transition_probabilities.T)
    # all eigenvalues will be <= 1, and one will be =1
    stationary_dist = eigvecs[:, np.argmax(eigvals)]
    return stationary_dist / sum(stationary_dist)

def entropy_rate(transition_probabilities):
    stationary_dist = get_stationary_dist(transition_probabilities)
    return sum([prob * entropy(transition_probabilities[i]) 
                for i, prob in enumerate(stationary_dist)])

Generate some data to pass through the SPA:

In [111]:
## You can change these
ALPHABET_SIZE = 2
PEAK_PROB = 0.9
K = 5
N = 1_000_000
N_TEST = 10_000

In [112]:
# Build data array; feel free to ignore this code and just run the cell
transition_probabilities = np.eye(ALPHABET_SIZE) * PEAK_PROB + \
    (np.ones((ALPHABET_SIZE, ALPHABET_SIZE)) - np.eye(ALPHABET_SIZE)) * (1 - PEAK_PROB) / (ALPHABET_SIZE - 1)
start_prob = np.ones(ALPHABET_SIZE) / ALPHABET_SIZE

data = np.zeros(N, dtype=int)
for i in range(K):
    data[i] = sample_index_from_dist(start_prob)
for i in range(K,N):
    data[i] = sample_index_from_dist(transition_probabilities[data[i-K]])

In [113]:
sequence = Sequence(data[:-N_TEST])

#### Instance method: `train_on_block`

Use a block of data to update the SPA. If `include_prev_context` is
true, then this block is considered to be from the same sequence as
the previous. Otherwise, it is assumed to be a separate sequence, and
we return to the root of the LZ78 prefix tree.

It returns the self-entropy log loss incurred while processing this
sequence.

In [114]:
spa = LZ78SPA(ALPHABET_SIZE)

In [115]:
stdout.flush()
spa.train_on_block(sequence)

0.6277613794856698

#### Instance method: `compute_test_loss`
After training a SPA, you can compute the log loss of a test sequence.

In [116]:
stdout.flush()
spa.compute_test_loss(Sequence(data[-N_TEST:]), include_prev_context=True) / N_TEST

0.7026905466910377

#### Instance method: `get_normalized_log_loss`
Gets the normaliized self-entropy log loss incurred from training the SPA thus far.

In [117]:
spa.get_normalized_log_loss()

0.6277613794856698

#### Instance method: `compute_spa_at_current_state`
Computes the SPA for every symbol in the alphabet, using the LZ78 context reached at the end of parsing the last training block.

In this case, the method will return a two-element list, where the first element is the estimated probability that the next symbol is $0$ and the second is the estimated probability that the next symbol is $1$.

In [119]:
# One component "should" be 0.9 and the other "should" be 0.1, but this is
# not necessarily the case. e.g., if we are at the top of the LZ78 prefix tree
# or at a leaf, we can expect the SPA to be closer to [0.5, 0.5]
spa.compute_spa_at_current_state()

[0.8855421686746988, 0.1144578313253012]

#### Instance method: `to_bytes`
This works the same as the corresponding index of `CompressedSequence`; refer to the LZ78 Encoding part of the tutorial for more details.

The method `spa_from_bytes` reconstructs a SPA from its `bytes` representation.

### 3.2 Example: Text Generation

Let's use the LZ78 SPA to generate some text based on Sherlock Holmes novels.

This requires the `requests` library and an internet connection.
If you don't have either, you can perform the same experiment any text you'd like, including the lorem ipsum text from the beginning of this tutorial.
Just make sure you have enough training data (e.g., the Sherlock novel used for this example is 500 kB).

In [123]:
text = requests.get("https://www.gutenberg.org/cache/epub/1661/pg1661.txt").text

Let's define our own character map and filter the text based on this.

In [127]:
stdout.flush()
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ,?\n\"';:\t-_")
filtered_text = charmap.filter_string(text)

Next, train the SPA.

In [179]:
stdout.flush()
spa = LZ78SPA(charmap.alphabet_size(), gamma=0.2)
spa.train_on_block(Sequence(filtered_text, charmap=charmap))

3.7578532624402827

#### Instance method: `generate_data`
Generates a sequence of data, using temperature and top-k sampling (see
the "Experiments" section of [Sagan and Weissman 2024] for more details).

Inputs:
- **len**: number of symbols to generate
- **min_context**: the SPA tries to maintain a context of at least a
    certain length at all times. So, when we reach a leaf of the LZ78
    prefix tree, we try traversing the tree with different suffixes of
    the generated sequence until we get a sufficiently long context
    for the next symbol.
- **temperature**: a measure of how "random" the generated sequence is. A
    temperature of 0 deterministically generates the most likely
    symbols, and a temperature of 1 samples directly from the SPA.
    Temperature values around 0.1 or 0.2 function well.
- **top_k**: forces the generated symbols to be of the top_k most likely
    symbols at each timestep.
- **seed_data**: you can specify that the sequence of generated data
be the continuation of the specified sequence.

Returns a tuple of the generated sequence and that sequence's log loss,
or perplexity.

Errors if the SPA has not been trained so far, or if the seed data is
not over the same alphabet as the training data.

In [182]:
(generated, loss) = spa.generate_data(
    500,
    min_context=5,
    temperature=0.1,
    top_k=5,
    seed_data=Sequence("This ", charmap=charmap)
)
generated = generated.get_data()
generated = "This " + generated

Right now, the generated text will not appear very high-quality for a few reasons:
1. We don't have a lot of training data. LZ78-based algorithms generally take a lot of data to train; empirically, the SPA does better with many megabytes of data, at least.
2. The `min_context` parameter is being used very heuristically, and could potentially be forcing us to the bottom of the LZ78 tree, where we don't have a lot of data, making the SPA closer to uniform. If you see long gibberish strings, try decreasing `min_context`.

In [183]:
for i in range(0, len(generated), 80):
    print(generated[i:i+80])

This is money father which had been out of the coronet into the advertisedm heir
a cotnsi a cloise to the door opf oened away to the stablise of the street. Suim
oddnsedt f the straini thng to him, and the chiarg we a rsoepnal tdo he stone in
 the conversa to
imthe sightnal there was a very sharpl iexn this morning I am a
lfll that the contrasnge for me to the contrary, fsaor the coronet into the adve
rtisedm heira cotmmchent. I saw himself interest in the colour?l own little tro 
kng pow what is not be ua
